In [1]:
from __future__ import print_function, division
from neuralnilm.source import RandomSegments, RandomSegmentsInMemory
import matplotlib.pyplot as plt
import numpy as np
from sys import stdout

In [2]:
source = RandomSegmentsInMemory(
    filename='/data/mine/vadeec/merged/ukdale.h5',
    target_appliance='fridge freezer',
    window=("2014-10-25", "2014-10-27"),
    seq_length=512,
    train_buildings=[1],
    validation_buildings=[1],
    n_seq_per_batch=8,
    standardise_input=True,
    standardise_targets=True,
    ignore_incomplete=True)

In [3]:
source.start()

In [8]:
for i in range(100):
    print("\r", i, end="")
    stdout.flush()
    X, y = source.queue.get()

 99

In [11]:
X, y = source.queue.get(timeout=5)

In [12]:
fig, axes = plt.subplots(nrows=2, sharex=True)
axes[0].plot(X[0, :, 0])
axes[1].plot(y[0, :, 0])
plt.show()